<a href="https://colab.research.google.com/github/goktugin/skills-introduction-to-github/blob/main/Filtering_SNPs_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import defaultdict
from scipy.stats import f_oneway
import numpy as np

!pip install cyvcf2
from cyvcf2 import VCF
import os
import subprocess
import re
import time

!apt-get update
!apt-get install -y bcftools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/

In [2]:
# .vcf.gz dosyasının tam URL'sini buraya yazın
vcf_gz_url = "ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"
chromosome = re.search(r'\bchr[0-9XYM]+', vcf_gz_url).group()

# .vcf.gz.tbi dosyasının URL'si
tbi_url = vcf_gz_url + ".tbi"

# Dosya adlarını URL'lerden çıkaralım (daha temiz bir kod için)
vcf_gz_filename = vcf_gz_url.split("/")[-1]
tbi_filename = tbi_url.split("/")[-1]

print(f".vcf.gz dosyası indiriliyor: {vcf_gz_filename}")
!wget {vcf_gz_url}

print(f"\n.tbi dosyası indiriliyor: {tbi_filename}")
!wget {tbi_url}

print("\nİndirme işlemleri tamamlandı. Dosyalar kontrol ediliyor:")
!ls -lh {vcf_gz_filename} {tbi_filename}


# İndirilecek VCF dosyası
vcf_file_path = f"ALL.{chromosome}.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"
# Panel dosyasının URL'si ve Colab'e indirilecek adı
panel_url = "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel"
panel_file_path = "integrated_call_samples_v3.20130502.ALL.panel" # wget ile indirilecek dosya adı

print(f"Panel dosyası indiriliyor: {panel_file_path}")
!wget -O {panel_file_path} {panel_url} # -O ile dosya adını belirtiyoruz
print("\nİndirme tamamlandı.")
!ls -lh {panel_file_path} # Dosyanın indiğini ve boyutunu kontrol et


try:
    # Panel dosyası genellikle tab ile ayrılmıştır ve ilk satır başlıktır. Sütun adları: 'sample', 'pop', 'super_pop', 'gender'
    panel_df = pd.read_csv(panel_file_path, sep='\t')

    # VCF dosyasındaki örnek ID'lerini alalım (cyvcf2 kullanarak)
    vcf_reader_for_samples = VCF(vcf_file_path) #Dosyayı okumaya hazır bir nesne (vcf_reader_for_samples) oluşturulmuş olur.
    vcf_samples = vcf_reader_for_samples.samples #.samples özelliği, VCF dosyasındaki bireylerin/örneklerin isimlerinin bir listesini döndürür.
    vcf_reader_for_samples.close() # Okuyucuyu kapatalım

    # Paneldeki örnekleri VCF'dekilerle filtreme
    panel_df_filtered = panel_df[panel_df['sample'].isin(vcf_samples)]

    # 'sample' ID'lerini 'super_pop' kodlarına eşleyen bir sözlük oluşturalım. Yani {'Sample1': 'EUR', 'Sample2': 'AFR', ...} gibi bir sonuç
    sample_to_pop = pd.Series(panel_df_filtered.super_pop.values, index=panel_df_filtered['sample']).to_dict()

    if not sample_to_pop:
        print("UYARI: 'sample_to_pop' sözlüğü boş. Panel dosyası içeriği veya VCF örnekleriyle eşleşme kontrol edilmeli.")
        print(f"Panel dosyasında bulunan örnek sayısı: {len(panel_df)}")
        print(f"VCF dosyasında bulunan örnek sayısı: {len(vcf_samples)}")
        print(f"Panelde VCF ile eşleşen örnek sayısı: {len(panel_df_filtered)}")

    print(f"Panel dosyasından {len(sample_to_pop)} örnek için popülasyon bilgisi başarıyla yüklendi.")
    print("Örnek bir eşleşme (ilk 5):", dict(list(sample_to_pop.items())[:5]))
    print("Paneldeki benzersiz süper popülasyonlar:", panel_df_filtered['super_pop'].unique())

except FileNotFoundError:
    print(f"HATA: Panel dosyası bulunamadı: {panel_file_path}")
    print("Lütfen panel dosyasının doğru indirildiğinden ve 'panel_file_path' değişkeninin doğru olduğundan emin olun.")
    sample_to_pop = {}
except KeyError as e:
    print(f"HATA: Panel dosyasında beklenen sütun bulunamadı: {e}")
    print("Panel dosyasının sütun adlarını ('sample', 'super_pop' vb.) ve formatını kontrol edin.")
    sample_to_pop = {}
except Exception as e:
    print(f"Panel dosyası işlenirken bir hata oluştu: {e}")
    sample_to_pop = {}



    # MAF (Minör Alel Frekansı) Filtrelemesi --> yaygın olan varyantları filtreliyoruz

filtered_vcf_path = f"ALL.{chromosome}.phase3_filtered_maf005.vcf.gz" # Yeni filtrelenmiş dosya adı

# ÖNEMLİ: bcftools'un MAF hesaplaması için INFO tag'ında AF (Allele Frequency) olması gerekir.
# 1000 Genomes VCF'lerinde bu genellikle vardı veya INFO/AC (Allele Count) ve INFO/AN (Allele Number) üzerinden hesaplanabilir.
# Eğer doğrudan MAF yoksa, 'AF > 0.05 && AF < 0.95' gibi bir filtre de kullanılabilir.
# En basit haliyle AF (Alternatif Alel Frekansı) üzerinden gidelim:
print(f"MAF filtrelemesi başlıyor (AF > 0.05 ve AF < 0.95)...")
!bcftools view -i 'INFO/AF > 0.05 && INFO/AF < 0.95' {vcf_file_path} -Oz -o {filtered_vcf_path} # i --> include, INFO/AF --> INFO sütununda bulunan "AF"
# minör alel frekansı (MAF) en az 0.05 olan varyantları tutar.
# Yani hem çok nadir varyantları (MAF &lt; 0.05) hem de neredeyse popülasyonda sabitlenmiş
# (alternatif alelin frekansı > 0.95, dolayısıyla referans alelin frekansı &lt; 0.05, yani MAF &lt; 0.05) varyantları çıkarır.
# Kısacası, popülasyonda belirli bir düzeyde polimorfik (çeşitlilik gösteren) olan varyantları seçer.
print(f"Filtrelenmiş VCF dosyası oluşturuldu: {filtered_vcf_path}")

# Filtrelenmiş VCF için indeks oluşturmayı unutmayın (verimlilik artışı için)
print(f"Filtrelenmiş VCF için indeks oluşturuluyor...")
!bcftools index {filtered_vcf_path}
print(f"İndeks oluşturuldu: {filtered_vcf_path}.tbi")


# ----- GİRİŞ VE ÇIKIŞ DOSYA ADLARI -----
# MAF ile filtrelediğiniz VCF dosyasının adı (bir önceki adımdaki çıktı)
maf_filtered_vcf = f"ALL.{chromosome}.phase3_filtered_maf005.vcf.gz"

# PLINK işlemleri için kullanılacak geçici dosya ön ekleri
plink_input_prefix = f"{chromosome}_maf_filtered_for_plink"
plink_ld_pruning_prefix = f"{chromosome}_ld_pruned_list"
plink_final_pruned_prefix = f"{chromosome}_maf_ld_pruned"

# LD Pruning sonrası ANOVA için kullanılacak nihai VCF dosyasının adı
ld_pruned_vcf_for_anova = f"ALL.{chromosome}.filtered_maf005_ldpruned.vcf.gz"
# ----- ----- ----- ----- ----- ----- -----

# 1. GÜNCELLENMİŞ PLINK Kurulumu
print("PLINK indiriliyor ve ayarlanıyor...")
# Önceki olası kalıntıları temizleyelim (daha güvenli bir kurulum için)
!rm -f plink plink.zip plink.* 큰*.* *.log # plink, plink.zip ve PLINK ile gelen diğer dosyaları sil

# GÜNCELLENMİŞ İNDİRME LİNKİ:
plink_download_url = "https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20231211.zip"
!wget {plink_download_url} -O plink.zip

# wget'in başarılı olup olmadığını kontrol edelim (dosya boyutu > 0 olmalı)
if os.path.exists("plink.zip") and os.path.getsize("plink.zip") > 0:
    print("PLINK zip dosyası başarıyla indirildi.")
    !unzip -o plink.zip # -o ile üzerine yazma komutu (overwrite)
    print("\nZip'ten çıkarılan dosyalar:")
    !ls -l # Zip'ten çıkan dosyaları listele, 'plink' adında bir dosya görmeliyiz

    if os.path.isfile("plink"):
        !chmod +x plink
        print("PLINK başarıyla ayıklandı ve çalıştırılabilir yapıldı.")
    else:
        print("HATA: PLINK çalıştırılabilir dosyası ('plink') zip'ten çıkarıldıktan sonra bulunamadı!")
        print("Lütfen yukarıdaki 'ls -l' çıktısını kontrol edin. Eğer farklı bir isimle çıkarıldıysa, kodda './plink' yerine o ismi kullanmanız gerekir.")
        raise FileNotFoundError("PLINK executable ('plink') not found after unzip.")
else:
    print(f"HATA: PLINK zip dosyası ({plink_download_url}) indirilemedi veya boş. Lütfen linki kontrol edin veya çalışan başka bir PLINK 1.9 indirme linki bulun.")
    raise FileNotFoundError("PLINK zip file download failed or the file is empty.")


    # ----- GİRİŞ VE ÇIKIŞ DOSYA ADLARI (bir öncekiyle aynı) -----
maf_filtered_vcf = f"ALL.{chromosome}.phase3_filtered_maf005.vcf.gz"
vcf_with_ids = f"ALL.{chromosome}.phase3_filtered_maf005.ids.vcf.gz"
plink_input_prefix = f"{chromosome}_maf_ids_for_plink"
plink_ld_pruning_prefix = f"{chromosome}_ld_pruned_list"
plink_final_pruned_prefix = f"{chromosome}_maf_ids_ld_pruned"
ld_pruned_vcf_for_anova = f"ALL.{chromosome}.filtered_maf005_ids_ldpruned.vcf.gz"
plink_executable = "./plink"
# ----- ----- ----- ----- ----- ----- ----- ----- -----

def run_command(cmd_list, log_prefix_for_error="cmd"):
    """Yardımcı fonksiyon: Komut çalıştırır, başarı/hata durumunu yönetir."""
    print(f"Komut çalıştırılıyor: {' '.join(cmd_list[:4])} ...") # Komutun başını göster
    log_file = f"{log_prefix_for_error}.log"
    # subprocess.run'dan önce log dosyasını silmeyin, PLINK kendisi oluşturur/üzerine yazar

    result = subprocess.run(cmd_list, capture_output=True, text=True, check=False)

    if result.returncode == 0:
        print(f"BAŞARILI: Komut tamamlandı. (Log: {log_file if os.path.exists(log_file) else 'oluşturulmadı'})")
        # Başarılıysa sadece kısa bir STDERR özeti (uyarılar için)
        if result.stderr.strip():
            print("  PLINK STDERR (Uyarılar olabilir):")
            # Sadece ilk birkaç satırı veya önemli uyarıları göster
            stderr_lines = result.stderr.strip().split('\n')
            for line in stderr_lines[:5]: # İlk 5 satır
                print(f"    {line}")
            if len(stderr_lines) > 5:
                print("    ...")
        return result.stdout # Başarılı durumda STDOUT'u döndür (işlemek için)
    else:
        print(f"HATA: Komut başarısız oldu. Dönüş kodu: {result.returncode}")
        print("  --- PLINK STDOUT ---")
        print(result.stdout)
        print("  --- PLINK STDERR ---")
        print(result.stderr)
        if os.path.exists(log_file):
            print(f"\n  --- {log_file} içeriği ---")
            with open(log_file, 'r') as f:
                print(f.read())
            print(f"  --- {log_file} sonu ---")
        raise RuntimeError(f"Komut başarısız: {' '.join(cmd_list[:3])}...")

# --- Adım 0: Dosya Kontrolleri (kısaltılmış) ---
if not (os.path.isfile(plink_executable) and os.access(plink_executable, os.X_OK)):
    raise FileNotFoundError(f"PLINK executable not found or not executable: {plink_executable}")
if not os.path.exists(maf_filtered_vcf):
    raise FileNotFoundError(f"Input VCF file not found: {maf_filtered_vcf}")
print("PLINK ve girdi VCF dosyası bulundu.")

# --- Adım 1: bcftools annotate
print(f"\nAdım 1: '{maf_filtered_vcf}' dosyasındaki eksik ID'ler bcftools annotate ile dolduruluyor...")
bcftools_annotate_cmd = ["bcftools", "annotate", "--set-id", "+'%CHROM:%POS:%REF:%FIRST_ALT'", maf_filtered_vcf, "-Oz", "-o", vcf_with_ids]
run_command(bcftools_annotate_cmd, "bcftools_annotate")
!bcftools index {vcf_with_ids}
print(f"{vcf_with_ids} dosyası oluşturuldu ve indekslendi.")

# --- Adım 2: VCF to BED (PLINK) ---
print(f"\nAdım 2: '{vcf_with_ids}' dosyası PLINK BED formatına dönüştürülüyor...")
plink_cmd_step2 = [plink_executable, "--vcf", vcf_with_ids, "--allow-extra-chr", "--make-bed", "--out", plink_input_prefix]
stdout_step2 = run_command(plink_cmd_step2, plink_input_prefix)
# PLINK'in yüklediği varyant sayısını STDOUT'tan veya log'dan alabiliriz (isteğe bağlı)
variants_loaded_match = re.search(r"(\d+) variants loaded from .bim file", stdout_step2)
if variants_loaded_match:
    print(f"  {variants_loaded_match.group(1)} varyant PLINK BED formatına yüklendi.")
print(f"PLINK BED formatına dönüştürme başarılı.")

# LD Pruning, yüksek LD'de olan SNP gruplarından sadece bir veya birkaç temsilci SNP'yi tutarak diğerlerini veri setinden çıkarır. Bu sayede, PCA için daha "bağımsız" ve daha az sayıda SNP içeren bir set elde edilir, bu da daha güvenilir ve yorumlanabilir popülasyon yapısı sonuçları verir.
# --- Adım 3: LD Pruning (PLINK) ---
window_size_snps = 50; step_size_snps = 5; r2_threshold = 0.2
print(f"\nAdım 3: LD Pruning işlemi başlıyor...")
plink_cmd_step3 = [plink_executable, "--bfile", plink_input_prefix, "--allow-extra-chr", "--indep-pairwise", str(window_size_snps), str(step_size_snps), str(r2_threshold), "--out", plink_ld_pruning_prefix]
stdout_step3 = run_command(plink_cmd_step3, plink_ld_pruning_prefix)
pruned_match = re.search(r"Pruning complete.  (\d+) of \d+ variants removed", stdout_step3)
if pruned_match:
    print(f"  LD Pruning tamamlandı. {pruned_match.group(1)} varyant çıkarıldı.")
else: # Logdan okumayı deneyebiliriz veya sadece genel mesaj
    print(f"  LD Pruning tamamlandı. Ayrıntılar için {plink_ld_pruning_prefix}.log dosyasına bakınız.")
print(f"LD Pruning başarılı.")

# --- Adım 4: Budanmış Seti Oluşturma (PLINK) ---
print(f"\nAdım 4: Budanmış SNP listesi kullanılarak yeni PLINK dosyası oluşturuluyor...")
plink_cmd_step4 = [plink_executable, "--bfile", plink_input_prefix, "--allow-extra-chr", "--extract", f"{plink_ld_pruning_prefix}.prune.in", "--make-bed", "--out", plink_final_pruned_prefix]
stdout_step4 = run_command(plink_cmd_step4, plink_final_pruned_prefix)
variants_remaining_match = re.search(r"--extract: (\d+) variants remaining", stdout_step4)
if variants_remaining_match:
    print(f"  {variants_remaining_match.group(1)} SNP ile yeni PLINK seti oluşturuldu.")
print(f"Yeni budanmış PLINK dosyası başarıyla oluşturuldu.")

# --- Adım 5: VCF'ye Geri Dönüşüm (PLINK) ---
output_vcf_prefix_for_plink = ld_pruned_vcf_for_anova.replace(".vcf.gz", "")
print(f"\nAdım 5: Son budanmış PLINK seti VCF formatına dönüştürülüyor: {ld_pruned_vcf_for_anova}")
plink_cmd_step5 = [plink_executable, "--bfile", plink_final_pruned_prefix, "--allow-extra-chr", "--recode", "vcf-iid", "bgz", "--out", output_vcf_prefix_for_plink]
run_command(plink_cmd_step5, output_vcf_prefix_for_plink)
if os.path.exists(f"{output_vcf_prefix_for_plink}.vcf.gz"):
    if f"{output_vcf_prefix_for_plink}.vcf.gz" != ld_pruned_vcf_for_anova:
         os.rename(f"{output_vcf_prefix_for_plink}.vcf.gz", ld_pruned_vcf_for_anova)
print(f"VCF formatına dönüştürme başarılı.")

# --- Adım 6: İndeksleme (bcftools) ---
print(f"\nAdım 6: {ld_pruned_vcf_for_anova} dosyası bcftools ile indeksleniyor...")
!bcftools index {ld_pruned_vcf_for_anova}
print(f"İndeksleme başarılı.")

print(f"\nLD Pruning tamamlandı! ANOVA analizi için kullanılacak dosya: {ld_pruned_vcf_for_anova}")
!ls -lh {ld_pruned_vcf_for_anova}* # Son dosyayı ve indeksini göster


# ----- ANOVA İÇİN GİRDİ DOSYASI -----
# LD Pruning sonrası oluşan VCF dosyasının adı:
vcf_file_path = f"ALL.{chromosome}.filtered_maf005_ids_ldpruned.vcf.gz"
# ----- ----- ----- ----- ----- -----

# sample_to_pop sözlüğünün ve diğer gerekli değişkenlerin
# (min_samples_per_pop_for_calc, top_n_snps_to_select)
# bir önceki hücrelerden yüklendiğini varsayıyoruz.
# Eğer tanımlı değillerse, burada tekrar tanımlamanız gerekir:

if 'sample_to_pop' not in locals() or not sample_to_pop:
    print("HATA: 'sample_to_pop' sözlüğü bulunamadı. Lütfen popülasyon bilgilerini yükleme adımını tekrar çalıştırın.")
    raise NameError("'sample_to_pop' sözlüğü tanımlı değil veya boş.")

if 'min_samples_per_pop_for_calc' not in locals():
    min_samples_per_pop_for_calc = 5 # Popülasyon başına ANOVA için min örnek


# --- ANOVA Analiz Kodu ---
try:
    vcf_reader = VCF(vcf_file_path)
except OSError as e:
    print(f"HATA: VCF dosyası ({vcf_file_path}) veya indeksi bulunamadı/okunamadı: {e}")
    raise

samples_in_vcf = vcf_reader.samples
pop_to_samples = defaultdict(list) #Amacımız, her bir süper popülasyon kodunu (örn: 'EUR', 'AFR') anahtar olarak ve o popülasyona ait örnek ID'lerinin bir listesini de değer olarak tutan bir sözlük oluşturmaktır. pop_to_samples bu sözlük olacaktır.
for sample_id, super_pop_code in sample_to_pop.items():
    if sample_id in samples_in_vcf:
         pop_to_samples[super_pop_code].append(sample_id)

sample_indices = {sample_name: i for i, sample_name in enumerate(samples_in_vcf)}
all_calculated_snps_with_pvalues = [] #ANOVA analizi sırasında, her bir SNP için bir p-değeri hesaplayacağız. Eğer bu p-değeri geçerliyse (NaN değilse), o SNP'ye ait bilgileri (kromozom, pozisyon, ID, p-değeri ve ANOVA'ya dahil edilen popülasyonlar) bir demet (tuple) olarak bu listeye ekleyeceğiz. Döngü tamamlandığında, bu liste, p-değeri hesaplanabilmiş tüm SNP'leri içerecektir. Daha sonra bu listeyi p-değerine göre sıralayıp en iyi N tanesini seçeceğiz.
processed_variants_count = 0 #VCF dosyasındaki varyantlar (SNP'ler) üzerinde dönerken, o ana kadar kaç tane varyantın işlendiğini (yani döngünün kaçıncı adımında olduğumuzu) saymak için kullanılacak bir sayaç
total_variants_in_file = 0 # VCF dosyasındaki toplam varyant satırı sayısını (yorum satırları ve başlık hariç) saymak için kullanılır. Döngü her bir varyant için çalıştığında bu sayaç artırılır. Analiz bittikten sonra, VCF dosyasında toplam kaç varyant olduğunu ve bunlardan kaçının işlendiğini raporlamak için kullanılır.

if pop_to_samples:
    print(f"ANOVA analizi '{vcf_file_path}' dosyası üzerinde başlıyor...")
    print(f"Popülasyon başına ANOVA için minimum örnek sayısı: {min_samples_per_pop_for_calc}")
    print(f"Analiz sonunda bütün SNP'ler seçilecektir.")
    start_time = time.time()

    for variant in vcf_reader: # Bu döngü şimdi çok daha hızlıdır
        total_variants_in_file += 1
        processed_variants_count += 1

        if processed_variants_count % 2000 == 0: # İlerleme bildirimi
            print(f"{processed_variants_count} varyant işlendi...")

        if not (len(variant.ALT) == 1 and len(variant.REF) == 1 and len(variant.ALT[0]) == 1):
            continue

        # Her yeni varyant için, ANOVA'ya girecek genotip değerlerini ve
        # bu ANOVA'ya dahil edilecek popülasyonları saklamak üzere boş listeler oluşturulur.
        pop_genotype_values_for_anova = []
        populations_in_anova = []

        # 'pop_to_samples' sözlüğü üzerinde döngüye girilir. Bu sözlük, her bir süper popülasyon kodunu ('pop_code')
        # o popülasyona ait örnek ID'lerinin bir listesine ('pop_sample_list') eşler.
        # Örn: pop_code = 'EUR', pop_sample_list = ['HG00096', 'HG00097', ...]
        for pop_code, pop_sample_list in pop_to_samples.items():
            # O anki popülasyona ('pop_code') ait olup, aynı zamanda VCF dosyamızda da bulunan
            # (yani 'sample_indices' sözlüğünde anahtarı olan) örnek ID'lerini seçer.
            current_pop_samples_in_vcf = [s_id for s_id in pop_sample_list if s_id in sample_indices]

            # Eğer bu popülasyondaki (VCF'de de bulunan) örnek sayısı, belirlediğimiz minimum eşikten
            # ('min_samples_per_pop_for_calc', örn: 5) az ise, bu popülasyon bu SNP için ANOVA'ya
            # dahil edilmez ve döngünün bir sonraki popülasyonuna geçilir ('continue').
            # Bu, çok az bireye sahip grupların istatistiksel sonuçları aşırı etkilemesini önler.
            if len(current_pop_samples_in_vcf) < min_samples_per_pop_for_calc:
                continue

            # Bu popülasyondaki ('current_pop_samples_in_vcf') bireylerin, o anki varyant için
            # sahip oldukları alternatif alel sayılarını toplayacağımız boş bir liste.
            individual_alt_allele_counts_in_pop = []

            # Popülasyondaki her bir örnek ('sample_name') için döngüye girilir.
            for sample_name in current_pop_samples_in_vcf:
                idx = sample_indices[sample_name]  # Örnek ID'sinin VCF'deki sayısal indeksini alır.
                genotype = variant.genotypes[idx]  # O örneğin, o anki varyanttaki genotipini alır.
                                                   # genotype bir listedir: [alel1_indeksi, alel2_indeksi, faz_bilgisi]
                                                   # alel_indeksi: 0 (referans alel), 1 (ilk alternatif alel), -1 (eksik veri).

                alt_count_for_sample = 0     # Bu birey için o anki SNP'deki alternatif alel sayısı.
                valid_alleles_for_sample = 0 # Bu birey için geçerli (eksik olmayan) alel sayısı.

                # Bireyin ilk alelini kontrol et:
                if genotype[0] != -1:  # Eğer alel eksik veri (-1) değilse:
                    alt_count_for_sample += genotype[0] # alel1_indeksi (0 veya 1) eklenir.
                    valid_alleles_for_sample += 1       # Geçerli alel sayısını artır.

                # Bireyin ikinci alelini kontrol et:
                if genotype[1] != -1:  # Eğer alel eksik veri (-1) değilse:
                    alt_count_for_sample += genotype[1] # alel2_indeksi (0 veya 1) eklenir.
                    valid_alleles_for_sample += 1       # Geçerli alel sayısını artır.

                # Eğer bireyin en az bir aleli geçerliyse (yani genotip tamamen ' ./. ' değilse),
                # hesaplanan alternatif alel sayısını (0, 1 veya 2 olabilir) listeye ekle.
                if valid_alleles_for_sample > 0 :
                    individual_alt_allele_counts_in_pop.append(alt_count_for_sample)

            # Eğer bu popülasyondan, minimum örnek sayısını ('min_samples_per_pop_for_calc')
            # karşılayacak kadar bireyden genotip verisi (alternatif alel sayısı) toplayabildiysek:
            if len(individual_alt_allele_counts_in_pop) >= min_samples_per_pop_for_calc:
                # Bu popülasyonun genotip değerlerini (bir NumPy dizisi olarak) ANOVA'ya girecek
                # genel listeye ('pop_genotype_values_for_anova') ekle.
                pop_genotype_values_for_anova.append(np.array(individual_alt_allele_counts_in_pop))
                # Bu popülasyonun kodunu ('pop_code') da ANOVA'ya dahil edilen popülasyonlar
                # listesine ('populations_in_anova') ekle.
                populations_in_anova.append(pop_code)

        # Tüm popülasyonlar için veri toplama işlemi bittikten sonra,
        # ANOVA testi yapabilmek için en az 2 popülasyon grubundan veri toplamış olmamız gerekir.
        if len(pop_genotype_values_for_anova) >= 2:
            try:
                # scipy.stats.f_oneway fonksiyonu ile tek yönlü ANOVA testi yapılır.
                # '*' operatörü, 'pop_genotype_values_for_anova' listesindeki her bir NumPy dizisini
                # (yani her bir popülasyonun alternatif alel sayıları listesini) fonksiyona
                # ayrı bir argüman olarak gönderir. Her bir argüman, ANOVA'da bir grup olarak kabul edilir.
                # Fonksiyon, F istatistiğini ve p-değerini döndürür.
                f_stat, p_value = f_oneway(*pop_genotype_values_for_anova)

                # Eğer hesaplanan p-değeri geçerli bir sayı ise (NaN - Not a Number - değilse):
                if not np.isnan(p_value):
                    # O anki varyantın Kromozom, Pozisyon, ID'si (bcftools ile atanan),
                    # hesaplanan p-değeri ve bu ANOVA'ya dahil edilen popülasyonların listesini
                    # bir demet (tuple) olarak 'all_calculated_snps_with_pvalues' listesine ekle.
                    all_calculated_snps_with_pvalues.append((variant.CHROM, variant.POS, variant.ID, p_value, populations_in_anova))
            except Exception as e:
                # ANOVA hesaplaması sırasında bir hata oluşursa (örn: tüm gruplardaki tüm değerler aynıysa
                # ve varyans sıfırsa, f_oneway uyarı verebilir veya hata fırlatabilir),
                # bu hatayı sessizce atla ('pass') ve bir sonraki varyanta geç.
                # İstenirse, hata ayıklama için buraya print(e) eklenebilir.
                pass

    end_time = time.time()
    total_elapsed_time = end_time - start_time
    print(f"\nANOVA hesaplamaları tamamlandı.")
    print(f"Toplam {processed_variants_count} varyant (dosyadaki toplam: {total_variants_in_file}) işlendi.")
    print(f"Toplam geçen süre: {total_elapsed_time:.2f} saniye.")
    print(f"{len(all_calculated_snps_with_pvalues)} SNP için geçerli p-değeri hesaplandı.")

    all_calculated_snps_with_pvalues.sort(key=lambda x: x[3]) # p-value (indeks 3) göre sırala

else:
    print("Popülasyon bilgileri ('pop_to_samples') yüklenemediği için ANOVA analizi yapılamıyor.")
vcf_reader.close()


if all_calculated_snps_with_pvalues:
    print("Veriler CSV dosyasına kaydediliyor...")

    # DataFrame oluşturmak için veriyi hazırla
    # 'populations_in_anova' listesini CSV'de daha okunabilir olması için bir stringe dönüştür
    data_for_df = []
    for item in all_calculated_snps_with_pvalues:
        chrom, pos, snp_id, p_val, pops_list = item
        # Popülasyon listesini virgülle ayrılmış bir stringe çevir
        pops_str = ",".join(pops_list)
        data_for_df.append([chrom, pos, snp_id, p_val, pops_str])

    # Pandas DataFrame oluştur
    # Sütun adlarını tanımla
    column_names = ['Chromosome', 'Position', 'SNP_ID', 'P_Value', 'Populations_in_ANOVA']
    df_results = pd.DataFrame(data_for_df, columns=column_names)

    # CSV dosyası olarak kaydet
    csv_file_name = f"anova_snps_{chromosome}_pvalues.csv"
    try:
        # index=False, DataFrame indeksinin CSV dosyasına yazılmasını engeller
        df_results.to_csv(csv_file_name, index=False)
        print(f"Sonuçlar başarıyla '{csv_file_name}' dosyasına kaydedildi.")
    except Exception as e:
        print(f"HATA: CSV dosyası kaydedilemedi: {e}")
else:
    print("Kaydedilecek veri bulunamadı (all_calculated_snps_with_pvalues listesi boş).")


# Genotiplerin okunacağı VCF dosyası (LD pruning sonrası oluşan dosya)
vcf_file_for_genotypes = f"ALL.{chromosome}.filtered_maf005_ids_ldpruned.vcf.gz"

# Çıktı csv dosyasının adı
output_csv_file = f"genotip_matrisi_tum_snpler_{chromosome}.csv" # Dosya adını güncelledim
# ----- ----- ----- ----- ----- ----- ----- ----- ----- -----

# print(f"{len(all_calculated_snps_with_pvalues)} SNP için genotip matrisi oluşturulacak.") # Bu satır artık geçerli değil
print(f"Girdi VCF dosyasındaki TÜM SNP'ler için genotip matrisi oluşturulacak: {vcf_file_for_genotypes}")


# 1. Artık önceden seçilmiş bir SNP seti kullanmıyoruz.
# selected_snp_ids_set = {snp_info[2] for snp_info in all_calculated_snps_with_pvalues} # KALDIRILDI/YORUMLANDI
# ordered_snp_ids_for_columns = [snp_info[2] for snp_info in all_calculated_snps_with_pvalues] # KALDIRILDI/YORUMLANDI
all_snp_ids_in_vcf_order = [] # VCF'deki sırayla SNP ID'lerini tutmak için yeni liste

# 2. VCF dosyasını aç ve örnek ID'lerini al
try:
    vcf_reader_gt = VCF(vcf_file_for_genotypes)
except OSError as e:
    print(f"HATA: VCF dosyası ({vcf_file_for_genotypes}) veya indeksi bulunamadı/okunamadı: {e}")
    raise

sample_ids = vcf_reader_gt.samples
print(f"{len(sample_ids)} örnek (birey) bulundu.")

# 3. Genotip verilerini toplamak için bir dictionary başlat
# İlk sütun olarak örnek ID'lerini ekleyelim
genotype_data_for_df = {'SampleID': sample_ids}

# 4. VCF dosyasını oku ve TÜM SNP'ler için genotipleri çıkar
processed_variants_count = 0 # İşlenen toplam varyant sayısı

print("VCF dosyası okunuyor ve tüm SNP'ler için genotipler çıkarılıyor...")
for variant in vcf_reader_gt:
    processed_variants_count += 1
    variant_id = variant.ID # bcftools annotate ile oluşturduğumuz ID (örn: CHR:POS:REF:ALT)
                           # Eğer ID yoksa veya farklı bir ID formatı kullanılıyorsa burayı ayarlamanız gerekebilir.
                           # ID yoksa (None ise) veya boşsa, benzersiz bir ID oluşturabilirsiniz:
    if variant_id is None or variant_id == ".":
        variant_id = f"{variant.CHROM}:{variant.POS}:{variant.REF}:{variant.ALT[0]}"


    # ARTIK SNP ID'SİNİ KONTROL ETMİYORUZ, TÜMÜNÜ ALIYORUZ
    # if variant_id in selected_snp_ids_set: # KALDIRILDI

    # Alternatif alel sayılarını al (0, 1, 2). Eksikse np.nan
    alt_allele_counts = []
    for gt_array in variant.genotypes:
        # gt_array = [allele_idx1, allele_idx2, is_phased]
        # allele_idx: 0=REF, 1=ALT1, -1=eksik
        allele1 = gt_array[0]
        allele2 = gt_array[1]

        if allele1 == -1 or allele2 == -1: # Eğer genotipin bir kısmı bile eksikse, tüm genotipi eksik sayalım
            alt_allele_counts.append(np.nan)
        else:
            # Biallelik varsayımıyla (REF=0, ALT=1), toplamları alternatif alel sayısını verir
            alt_allele_counts.append(allele1 + allele2)

    genotype_data_for_df[variant_id] = alt_allele_counts
    all_snp_ids_in_vcf_order.append(variant_id) # SNP ID'sini sırayla listeye ekle

    if processed_variants_count % 1000 == 0: # Her 1000 varyantta bir ilerleme bildir
        print(f"  {processed_variants_count} varyant işlendi...")

vcf_reader_gt.close()
print(f"VCF okuma tamamlandı. {processed_variants_count} adet varyant (SNP) için genotip verisi toplandı.")

# Artık seçilmiş SNP'lerle karşılaştırma yapmıyoruz.
# if found_selected_snps_in_vcf != len(selected_snp_ids_set):
#     print(f"UYARI: ANOVA ile seçilen {len(selected_snp_ids_set)} SNP'den sadece {found_selected_snps_in_vcf} tanesi VCF dosyasında bulundu!")
#     print("Bu durum, SNP ID eşleşmesinde bir sorun olduğuna veya VCF dosyasının beklenenden farklı olduğuna işaret edebilir.")


# 5. Pandas DataFrame oluştur
# Sütunların doğru sırada ('SampleID' ve ardından VCF'deki sırayla SNP'ler) olması için
final_columns_for_df = ['SampleID'] + all_snp_ids_in_vcf_order

try:
    genotype_df = pd.DataFrame(genotype_data_for_df)
    # Sütunları 'SampleID' ve ardından VCF'deki SNP sırasına göre ayarla
    genotype_df = genotype_df[final_columns_for_df]
except KeyError as e:
    print(f"DataFrame oluşturulurken sütun hatası: {e}")
    print("Muhtemelen 'all_snp_ids_in_vcf_order' ile 'genotype_data_for_df' arasında bir uyumsuzluk var.")
    print("DataFrame mevcut sütunlarla oluşturuluyor...")
    genotype_df = pd.DataFrame(genotype_data_for_df) # Ham haliyle oluştur


print(f"\nDataFrame oluşturuldu. Boyut: {genotype_df.shape[0]} satır, {genotype_df.shape[1]} sütun.")
print("İlk 5 satır ve ilk birkaç sütun:")
# Sütun sayısı 6'dan azsa hata vermemesi için min() kullandım
print(genotype_df.iloc[:5, :min(6, genotype_df.shape[1])])

# 6. DataFrame'i csv dosyasına kaydet
print(f"\nDataFrame '{output_csv_file}' dosyasına kaydediliyor...")
try:
    genotype_df.to_csv(output_csv_file, index=False)
    print(f"'{output_csv_file}' başarıyla kaydedildi!")
except Exception as e:
    print(f"CSV dosyasına kaydetme sırasında bir hata oluştu: {e}")




.vcf.gz dosyası indiriliyor: ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
--2025-05-22 17:20:06--  http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209774472 (200M) [application/x-gzip]
Saving to: ‘ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’

ALL.chr21.phase3_sh 100%[===================>] 200.06M  28.5MB/s    in 8.1s    

2025-05-22 17:20:15 (24.8 MB/s) - ‘ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’ saved [209774472/209774472]


.tbi dosyası indiriliyor: ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi
--2025-05-22 17:20:15--  http://ftp.1000genom